# 트위터 감성분석 예측모형

[트위터 감성 예측](nlp-twitter-ml.html)을 이어서 예측모형 개발을 계속 진행하는데 spaCy 처리작업에 시간이 많이 소요되어 데이터를 1,000건으로 줄인다.

In [2]:
import pandas as pd

df = pd.read_csv("data/twitter_sentiment_train.csv", encoding = "ISO-8859-1")
smpl_df = df.sample(1000, random_state = 77777)
smpl_df.columns = ["item_id", "sentiment", "text"]
smpl_df.shape

(1000, 3)

# 품사(POS) 태깅

한글의 경우 형태소로 쪼개서 토큰화하는 이유는 한국어에는 조사가 존재하여 영어에서 효과가 검증된 기법을 사용할 수 없다는 점이다.

["Python으로 딥러닝하기 | 자연어 처리 1. 토크나이징"](https://everyday-deeplearning.tistory.com/entry/Python으로-딥러닝하기-자연어-처리-1-토크나이징)에 나온 사례를 바탕으로 한 문장을 토큰화하는 과정을 비교해보자. 음절은 문자 단위, 어절은 단어 단위 (쉽게 띄어쓰기 단위), 형태소는 의미를 가진 가장 작은 단위로 쪼개는 것이다.

- 예시 문장) "한글 자연어 처리 공부는 어렵다"
    - 음절: 한, 글, 자, 연, 어, 처, 리, 공, 부, 는, 어, 렵, 다
    - 어절: 한글, 자연어, 처리, 공부는, 어렵다
    - 형태소: 한글, 자연어, 처리, 공부, 는, 어렵, 다

텍스트 품사(POS) 태깅을 위해서 `NLTK`를 많이 사용했으나, 최근에는 spaCy로 넘어가고 있는 추세다. 
아나콘다를 기반으로 작업을 진행하는 경우 다음과 같이 `conda install` 명령어를 사용해서 spacy를 설치하면 된다.
그리고 나서 영어 사전이 필요하니 `python -m spacy` 명령어로 영어를 사용할 수 있도록 설치한다.

- `$ conda install -c conda-forge spacy` # conda config --add channels conda-forge
- `$ python -m spacy download en`

## spaCy 들어가며

영어 문장을 spaCy를 통해서 `Wall Street Journal just published an interesting piece on crypto currencies` 영어문장을 형태소분석 작업을 다음과 같이 수행할 수 있고, 이외에도 `token.dep_`, `oken.is_stop` 등도 확인할 수 있다.

In [16]:
import spacy

nlp = spacy.load("en_core_web_sm")

print("=="*40)
print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}".format(
    "Text",
    "Index",
    "Lemma",
    "PUNCT",
    "Alpha",
    "Shape",
    "POS",
    "TAG"))
print("=="*40)

doc = nlp("Wall Street Journal just published an interesting piece on crypto currencies")
for token in doc:
    print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}".format(
        token.text,
        token.idx,
        token.lemma_,
        token.is_punct,
        token.is_space,
        token.shape_,
        token.pos_,
        token.tag_
    ))

Text	Index	Lemma	PUNCT	Alpha	Shape	POS	TAG
Wall	0	Wall	False	False	Xxxx	PROPN	NNP
Street	5	Street	False	False	Xxxxx	PROPN	NNP
Journal	12	Journal	False	False	Xxxxx	PROPN	NNP
just	20	just	False	False	xxxx	ADV	RB
published	25	publish	False	False	xxxx	VERB	VBD
an	35	an	False	False	xx	DET	DT
interesting	38	interesting	False	False	xxxx	ADJ	JJ
piece	50	piece	False	False	xxxx	NOUN	NN
on	56	on	False	False	xx	ADP	IN
crypto	59	crypto	False	False	xxxx	ADJ	JJ
currencies	66	currency	False	False	xxxx	NOUN	NNS


# spaCy 병렬처리

[stackoverflow, "Applying Spacy Parser to Pandas DataFrame w/ Multiprocessing"](https://stackoverflow.com/questions/44395656/applying-spacy-parser-to-pandas-dataframe-w-multiprocessing)를 참조하여 Spacy Parser를 데이터프레임에 멀티프로세싱으로 처리할 수 있다.

In [18]:
tokens = []
lemma = []
pos = []

for doc in nlp.pipe(smpl_df['text'].astype('unicode').values, batch_size=50, n_threads=8):
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

smpl_df['text_tokens'] = tokens
smpl_df['text_lemma'] = lemma
smpl_df['text_pos'] = pos
smpl_df.head()

,item_id,sentiment,text,text_tokens,text_lemma,text_pos
16392,16404,0,... Football.... Its gone!,"[..., Football, ...., , Its, gone, !]","[..., football, ...., , -PRON-, go, !]","[PUNCT, NOUN, PUNCT, SPACE, DET, VERB, PUNCT]"
92027,92039,0,@bar_bar_ella Unfortunately the thumb drive cr...,"[@bar_bar_ella, Unfortunately, the, thumb, dri...","[@bar_bar_ella, unfortunately, the, thumb, dri...","[VERB, ADV, DET, NOUN, NOUN, VERB, PROPN, PROP..."
82269,82281,1,@ashleylynnangle Lol but that place is awesome...,"[@ashleylynnangle, Lol, but, that, place, is, ...","[@ashleylynnangle, Lol, but, that, place, be, ...","[PUNCT, PROPN, CCONJ, DET, NOUN, VERB, ADJ, PU..."
83920,83932,0,@cateedelaloye miss you,"[@cateedelaloye, miss, you]","[@cateedelaloye, miss, -PRON-]","[NOUN, VERB, PRON]"
37637,37649,1,@allbloominwrong That is so sweet. Please say ...,"[@allbloominwrong, That, is, so, sweet, ., Ple...","[@allbloominwrong, that, be, so, sweet, ., ple...","[PUNCT, DET, VERB, ADV, ADJ, PUNCT, INTJ, VERB..."
